# Binary Search

**Thought:**

The core idea here is to make the search space become smaller dramatically, the normal way is to make the search space become only halp of the original one.

The prunning depends on the rule governning the array, like sorted.

### Search space reduction

A good problem for search in a sorted matrix, with shows how to reduce the search space wisely. [link](https://leetcode.com/articles/search-a-2d-matrix-ii/)

In [1]:
class Solution:
    def searchMatrix(self, matrix, target):
        # an empty matrix obviously does not contain `target` (make this check
        # because we want to cache `width` for efficiency's sake)
        if len(matrix) == 0 or len(matrix[0]) == 0:
            return False

        # cache these, as they won't change.
        height = len(matrix)
        width = len(matrix[0])

        # start our "pointer" in the bottom-left
        row = height-1
        col = 0

        while col < width and row >= 0:
            if matrix[row][col] > target:
                row -= 1
            elif matrix[row][col] < target:
                col += 1
            else: # found it
                return True
        
        return False

### Make use of binary tree features

For complete binary tree, here is a [problem](https://leetcode.com/problems/count-complete-tree-nodes/description/)

In [2]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def countNodes(self, root):
        """
        :type root: TreeNode
        :rtype: int
        """
        if not root:
            return 0
        ans = 0
        layer = 0
        def helper(root):
            layer = 0
            while root:
                layer += 1
                root = root.right
            return layer
        ans = 0
        while root:
            ans += 1
            if not root.left and not root.right:
                return ans
            if root.left and not root.right:
                return ans + 1
            leftlayer = helper(root.left)
            rightlayer = helper(root.right)
            if leftlayer == rightlayer:
                ans += 2**(rightlayer)-1
                root = root.left
            elif leftlayer > rightlayer:
                ans += 2**(leftlayer)-1
                root = root.right

### Use binary search to enhance dynamic programming

Problem [link](https://leetcode.com/problems/longest-increasing-subsequence/discuss/74824/JavaPython-Binary-search-O(nlogn)-time-with-explanation)

In [8]:
class Solution(object):
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n = len(nums)
        # dp[i]: for a sequence with length i+1, the last number will be dp[i]
        dp = [0]*n
        size = 0
        for num in nums:
            left, right = 0, size
            while left < right:
                mid = (left+right)//2
                if dp[mid] < num:
                    left = mid+1
                else:
                    right = mid
            dp[right] = num
            size = max(size, right+1)
        return size

Another [problem](https://leetcode.com/problems/russian-doll-envelopes/description/) with the same idea:

In [3]:
class Solution(object):
    def maxEnvelopes(self, envelopes):
        """
        :type envelopes: List[List[int]]
        :rtype: int
        """
        if not envelopes:
            return 0
        def check(outside, inside):
            return outside[1] > inside[1]
        n = len(envelopes)
        envelopes.sort(key=lambda v:(v[0], -v[1])) # The non-trival part is how to arrange the order by hight.
        dp = []
        for env in envelopes:
            if not dp:
                dp.append(env)
            size = len(dp)
            left, right = 0, size
            while left < right:
                mid = (left+right)//2
                if check(env, dp[mid]):
                    left = mid+1
                else:
                    right = mid  
            if left == size:
                dp.append(env)
            else:
                dp[right] = env
        return len(dp)

### Not only search by index

We can also search for a real value and then using binary seach to make it smaller and get the specific answer.

Here is a example about search the $k^{th}$ number among two arrays and the median. [link](https://leetcode.com/problems/median-of-two-sorted-arrays/description/)

In [1]:
class Solution(object):
    def kth(self,A,B,k):
        # Here it uses pigeon theory
        if not A:
            return B[k]
        if not B:
            return A[k]
        ia, ib = len(A)//2, len(B)//2
        ma, mb = A[ia], B[ib]
        
        if ia + ib <k:
            if ma > mb:
                return self.kth(A,B[ib+1:],k-ib-1)
            else:
                return self.kth(A[ia+1:],B,k-ia-1)
        else:
            if ma > mb:
                return self.kth(A[:ia],B,k)
            else:
                return self.kth(A,B[:ib],k)
    
    def findMedianSortedArrays(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: float
        """
        l = len(nums1) + len(nums2)
        if l%2:
            return self.kth(nums1,nums2,l//2)
        return (self.kth(nums1,nums2,l//2) + self.kth(nums1,nums2,l//2-1))/2.0

In [2]:
def median(A, B):
    m, n = len(A), len(B)
    if m > n:
        A, B, m, n = B, A, n, m
    if n == 0:
        raise ValueError

    imin, imax, half_len = 0, m, (m + n + 1) / 2
    while imin <= imax:
        i = (imin + imax) / 2
        j = half_len - i
        # here is a very good judgement for whether we are at the right position
        if i < m and B[j-1] > A[i]:
            # i is too small, must increase it
            imin = i + 1
        elif i > 0 and A[i-1] > B[j]:
            # i is too big, must decrease it
            imax = i - 1
        else:
            # i is perfect

            if i == 0: max_of_left = B[j-1]
            elif j == 0: max_of_left = A[i-1]
            else: max_of_left = max(A[i-1], B[j-1])

            if (m + n) % 2 == 1:
                return max_of_left

            if i == m: min_of_right = B[j]
            elif j == n: min_of_right = A[i]
            else: min_of_right = min(A[i], B[j])

            return (max_of_left + min_of_right) / 2.0

In [ ]:
# my answer, which is more complex because I didn't find the good stop condition, so I used the bisearch to how many number is smaller.
import bisect
class Solution(object):
    def findMedianSortedArrays(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: float
        """
        m = len(nums1)
        n = len(nums2)
        if m == 1 and n == 1:
            return float(nums1[0]+nums2[0])/2.0
        if m < n:
            nums1, nums2, m, n = nums2, nums1, n, m
        target = (m+n)//2
        even = False
        if (m+n)%2==0:
            even = True
        left, right = 0, m-1
        target2 = -1
        leftposition = True
        while left <= right:
            mid = (left+right)//2
            target2 = target -mid
            index2_left = bisect.bisect_left(nums2, nums1[mid])
            index2_right = bisect.bisect_right(nums2, nums1[mid])
            if index2_left <= target2 and target2 <= index2_right:
                if not even:
                    return nums1[mid]
                else:
                    if mid == 0:
                        return float(nums2[target2-1]+nums1[0])/2.0
                    elif target2==0:
                        return float(nums1[mid] +nums1[mid-1])/2.0
                    else:
                        return float(max(nums1[mid-1], nums2[target2-1])+nums1[mid])/2.0
            elif target2 > index2_right:
                left = mid + 1
                leftposition = False
            elif target2 < index2_left:
                right = mid - 1
                leftposition = True
        left, right = 0, n-1
        while left <= right:
            mid = (left+right)//2
            target1 = target -mid
            index1_left = bisect.bisect_left(nums1, nums2[mid])
            index1_right = bisect.bisect_right(nums1, nums2[mid])
            if index1_left <= target1 and target1 <= index1_right:
                if not even:
                    return nums2[mid]
                else:
                    if mid == 0:
                        return float(nums1[target1-1]+nums2[0])/2.0
                    elif target2==0:
                        return float(nums2[mid] +nums2[mid-1])/2.0
                    else:
                        return float(max(nums2[mid-1], nums1[target1-1])+nums2[mid])/2.0
            elif target1 > index1_right:
                left = mid + 1
            elif target1 < index1_left:
                right = mid - 1

Another problem of search kth number in a sorted matrix: [link](https://leetcode.com/problems/kth-smallest-element-in-a-sorted-matrix/description/)

In [3]:
class Solution(object):
    def kthSmallest(self, matrix, k):
        """
        :type matrix: List[List[int]]
        :type k: int
        :rtype: int
        """
        n = len(matrix)
        lo, hi = matrix[0][0], matrix[n-1][n-1]
        while lo < hi:
            mid = (lo + hi)/2
            count = sum(bisect.bisect_right(row, mid) for row in matrix)
            #print lo, hi, mid, count
            if count == k:
                break
            if count > k:
                hi = mid  
            else:
                lo = mid + 1
        mid = (lo+hi)/2
        ans = None
        for row in matrix:
            i = bisect.bisect_right(row, mid) - 1
            if i >= 0 and (ans is None or row[i] > ans):
                ans = row[i]
        return ans

**This is a very interesting approach to find solution:**

Here are other problems:

- Split Array Largest Sum: [link](https://leetcode.com/explore/learn/card/binary-search/146/more-practices-ii/1042/)
- Find K-th Smallest Pair Distance [link](https://leetcode.com/explore/learn/card/binary-search/146/more-practices-ii/1041/)
- K-th Smallest Prime Fraction [link](https://leetcode.com/problems/k-th-smallest-prime-fraction/description/)


#### Using binary search to find a interger answer unrelated to index of array

Problem [link](https://leetcode.com/problems/maximum-length-of-repeated-subarray/solution/)

In [1]:
# Dynmic programming:

class Solution(object):
    def findLength(self, A, B):
        """
        :type A: List[int]
        :type B: List[int]
        :rtype: int
        """
        m = len(A)
        n = len(B)
        dp = [[0]*(n+1) for _ in range(m+1)]
        ans = 0
        for i in range(m-1, -1, -1):
            for j in range(n-1, -1, -1):
                if A[i] == B[j]:
                    dp[j][i] = dp[j+1][i+1] + 1
                    ans = max(ans, dp[j][i])
        return ans

In [2]:
# Binary search:

class Solution(object):
    def findLength(self, A, B):
        """
        :type A: List[int]
        :type B: List[int]
        :rtype: int
        """
        if len(A) > len(B):
            return self.findLength(B, A)
        
        def check(length):
            lookup = set(tuple(A[i:i+length]) for i in range(len(A)-length+1))
            return any(tuple(B[j:j+length]) in lookup for j in range(len(B)-length+1))
        
        left, right = 1, len(A)+1
        while left < right:
            mid = (left+right)//2
            if check(mid):
                left = mid+1
            else:
                right = mid
        return left-1

### N sum

problem [link](https://leetcode.com/problems/4sum/description/):

In [38]:
class Solution(object):
    def fourSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[List[int]]
        """
        def binary2sums(nums, target, pre, res):
            n = len(nums)
            if n < 2:
                return
            left, right = 0, n-1
            while left < right:
                if nums[left] + nums[right] == target:
                    res.append(pre+[nums[left], nums[right]])
                    left += 1
                    right -= 1
                    while left < n:
                        if nums[left] == nums[left-1]:
                            left += 1
                        else:
                            break
                    while right > 0:
                        if nums[right] == nums[right+1]:
                            right -= 1
                        else:
                            break
                elif nums[left] + nums[right] < target:
                    left += 1
                else:
                    right -= 1
        def findNsums(nums, target, N, pre, res):
            n = len(nums)
            if N < 2 or N > n:
                return
            if N == 2:
                binary2sums(nums, target, pre, res)
            else:
                if target > nums[-1]*N:
                    return
                for i in range(n-N+1):
                    if nums[i]*N > target:
                        break
                    else:
                        if i == 0 or (i>0 and nums[i]!= nums[i-1]):
                            findNsums(nums[i+1:], target-nums[i], N-1, pre+[nums[i]], res)
        nums.sort()
        res = []
        findNsums(nums, target, 4, [], res)
        return res

In [39]:
so = Solution()

In [40]:
so.fourSum([1,0,-1,0,-2,2],0)

[[-2, -1, 1, 2], [-2, 0, 0, 2], [-1, 0, 0, 1]]

In [4]:
1e-9

1e-09

In [5]:
10**(-9)

1e-09

In [6]:
2e-9

2e-09

In [7]:
2**(-9)

0.001953125

In [8]:
2*10**(-9)

2e-09